In [1]:
#pip install yfinance

In [2]:
#pip install get-all-tickers

In [3]:
import yfinance as yf
import pandas as pd
import shutil
import os
import time
import glob
import smtplib
import ssl
from get_all_tickers import get_tickers as gt

In [4]:
tickers = ['^GSPC', 'XLK', 'BLOK', 'NLR', 'HSV.L', 'SOFI']
print("The amount of stocks chosen to observe: " + str(len(tickers)))

The amount of stocks chosen to observe: 6


In [5]:
#Purge- removes the filepath, for the indiv stocks, along with all the contents inside it then recreates the path for staging
shutil.rmtree("C:\\Users\\ckraft\\Desktop\\My Experiments\\Personal Use\\Portfolio")
os.mkdir("C:\\Users\\ckraft\\Desktop\\My Experiments\\Personal Use\\Portfolio")
#holds the amount of API calls we executed
Amount_of_API_Calls = 0 #while loop = responsible for storing historical data for each ticker on the list
Stock_Failure = 0 #to make sure to not linger too long on API calls where a stock ticker is struggling
Stocks_Not_Imported = 0 


i = 0 #iteration through the list of tickers
while (i < len(tickers)) and (Amount_of_API_Calls < 1800):
    try:
        stock = tickers[i]  #grabs the current stock ticker
        temp = yf.Ticker(str(stock))
        Hist_data = temp.history(period="max")  #put a timeframe ofr period!!! #(20 - 30 yrs)!!!!!!!!!!!
        #tells yfinance what kind of data we want about this stock 
        Hist_data.to_csv("C:\\Users\\ckraft\\Desktop\\My Experiments\\Personal Use\\Portfolio\\"+ stock +".csv")  #saves the historical data in csv format for further processing later
        time.sleep(2)  #pauses the loop for two seconds so to not cause issues for the Yahoo Finance's backend operations
        Amount_of_API_Calls += 1 
        Stock_Failure = 0
        i += 1  #iteration to the next ticker
    except ValueError:
        print("Yahoo Finance Backend Error, Attempting to Fix")  #an error occured on Yahoo Finance's backend. Try to retreive the data again.
        if Stock_Failure > 5:  #ok...move on to the next ticker if the current ticker fails more than 5 times
            i+=1
            Stocks_Not_Imported += 1
        Amount_of_API_Calls += 1
        Stock_Failure += 1
    
print("The amount of stocks we successfully imported: " + str(i - Stocks_Not_Imported))

The amount of stocks we successfully imported: 6


In [6]:
list_files = (glob.glob("C:\\Users\\ckraft\\Desktop\\My Experiments\\Personal Use\\Portfolio\\*.csv")) # Creates a list of all csv filenames in the stocks folder
new_data = [] #a 2D array to hold our stock name and OBV score
interval = 0  #for iteration
while interval < len(list_files):
    Data = pd.read_csv(list_files[interval]).tail(10)  #gets the most recent 10 days of trading for the current stock in iteration
    pos_move = []  #lists days of stock price increased
    neg_move = []  #lists days of stock price decreased
    OBV_Value = 0  #sets the initial OBV_Value to zero
    count = 0
    while (count < 10):  #most recent 10 trading days
        if Data.iloc[count,1] < Data.iloc[count,4]:  #true = if the stock increased in price
            pos_move.append(count)  #add the day to the pos_move list
        elif Data.iloc[count,1] > Data.iloc[count,4]:  #true = if the stock decreased in price
            neg_move.append(count)  #add the day to the neg_move list
        count += 1
    count2 = 0
    for i in pos_move:  #adds the volumes of positive days to OBV_Value, divides by opening price to normalize across all stocks
        OBV_Value = round(OBV_Value + (Data.iloc[i,5]/Data.iloc[i,1]))
    for i in neg_move:  #subtracts the volumes of negative days from OBV_Value, divide by opening price to normalize across all stocks
        OBV_Value = round(OBV_Value - (Data.iloc[i,5]/Data.iloc[i,1]))
    Stock_Name = ((os.path.basename(list_files[interval])).split(".csv")[0])  #gets the name of the current stock we are analyzing
    new_data.append([Stock_Name, OBV_Value])  #adds the stock name and OBV value to the new_data list
    interval += 1
df = pd.DataFrame(new_data, columns = ['Stock', 'OBV_Value'])  #creates a new dataframe from the new_data list
df["Stocks_Ranked"] = df["OBV_Value"].rank(ascending = False)  #rank the stocks by their OBV_Values
df.sort_values("OBV_Value", inplace = True, ascending = False)  #sort the ranked stocks
df.to_csv("C:\\Users\\ckraft\\Desktop\\My Experiments\\Personal Use\\Portfolio\\OnBalanceVolume_Ranked.csv", index = False)  # Save the dataframe to a csv without the index column
print("Your import has been published.")

Your import has been published.


In [7]:
#On Balance Volume (OBV)-- On-balance volume (OBV) is a technical trading momentum indicator 
#that uses volume flow to predict changes in stock price. Joseph Granville first developed 
#the OBV metric in the 1963 book Granville's New Key to Stock Market Profits.

In [8]:
#OBV formula --  is simply a running total of positive and negative volume. 
#A period's volume is positive when the close is above the prior close
#and is negative when the close is below the prior close.

#OBV = Previous OBV + Current Day's Volume. If the closing price of the asset is the same as the previous day's closing price:
#OBV = Previous OBV (+ 0) If the closing price of the asset is lower than the previous day's closing price:
#OBV = Previous OBV – Current Day's Volume.

Giving up send_request(...) after 11 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)'))))
error uploading: HTTPSConnectionPool(host='api.segment.io', port=443): Max retries exceeded with url: /v1/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))
